In [2]:
import tensorflow as tf
import numpy as np
import json
import pandas as pd
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import speech_recognition as sr
import pyttsx3
import subprocess as s
import datetime
from datetime import time
import os
import sqlite3
import nltk
import string
# nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


In [8]:
%%writefile content.json
{
  "intents": [
    {
"tag": "greeting",
"patterns": ["Hi?", "Hey", "How are you","hi there","hello"],
"responses": ["Hello", "hi,what i can help u","hii how are you"]
},
     {
"tag": "college name",
"patterns": ["what is the college name","about your college","name college"],
"responses": ["SNJB's Late Sau. Kantabai Bhavarlalji Jain College of Engineering,Chandwad"]
},
      
      {
"tag": "department name",
"patterns": ["Deparment name","stream name","field name","how many department college","count department name","available department"],
"responses": ["There are total six deparments in our college including Computer , Mechanical, Civil , E & TC , AIDS , MBA etc."]
},
       {
"tag": "department count,number",
"patterns": ["tell me count of ","stream name","field name","how many department college","count department name","available department"],
"responses": ["There are total six deparments in our college including Computer , Mechanical, Civil , E & TC , AIDS , MBA etc."]
},
      {
"tag": "vision and mission",
"patterns": ["Vision of Mission college","vision and mission of college "],
"responses": ["Vision of Our college is Transform young aspirant learners towards creativity and professionalism for societal growth through quality technical education.  and Mission is To share values, ideas, beliefs by encouraging faculties and students for welfare of society."]
},
      {
"tag": "year",
"patterns": ["established year of college","when college is started"],
"responses": ["2003"]
},
      {
"tag": "principal",
"patterns": ["principal name of college","about principal"],
"responses": ["Dr.M.D.Kokate is Principal of SNJB's KBJ College of Engineering Chandwad"]
},
      {
"tag": "library",
"patterns": ["library information","information about library","information of library"],
"responses": ["We have an excellent, attractive and pleasant ambiance library in the college. Library is located at first floor & Second floor of college building, having total area of 490 sq. mtr."]
},
      {
"tag": "Placement",
"patterns": ["placement information 2021 and 2022","about placement"],
"responses": ["in 2021 22 placement count was 38 candidates from computer department, in 2020 21 placement count was 48 cag"]
},
       {
"tag": "vice principal",
"patterns": ["name of vice principal","about vice principal"],
"responses": ["Sanghavi Sir is vice principle of SNJB's KBJ College of Engineering Chandwad"]
},
       {
"tag": "computer department lab",
"patterns": ["how many labs in computer department","Number of labs in computer department","count of computer depatment lab"],
"responses": ["Total 12 Labs are available in computer department"]
}
  ]
}

Overwriting content.json


In [12]:
with open("content.json") as content:
    data=json.load(content)

In [13]:
tags=[]

responses={}
patt=[]
filter_sentence = []
for intent in data1['intents']:
    responses[intent['tag']]=intent['responses']
    for lines in intent['patterns']:
        patt.append(lines)
        tags.append(intent['tag'])

# # stop_words = set(stopwords.words('english'))
# filter_sentence = [w for w in ntokens if not w.lower() in stop_words]
# for ntokens in patterns:
#     word_tokens = word_tokenize(ntokens)
#     filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
#     #with no lower case conversion
#     filtered_sentence = []
#     for w in word_tokens:
#         if w not in stop_words:
#             filtered_sentence.append(w)
#             patt.append(str(" ".join([c for c in filter_sentence])))
    
# patt
patt

['Hi?',
 'Hey',
 'How are you',
 'hi there',
 'hello',
 'what is the college name',
 'about your college',
 'name college',
 'Deparment name',
 'stream name',
 'field name',
 'how many department college',
 'count department name',
 'available department',
 'tell me count of ',
 'stream name',
 'field name',
 'how many department college',
 'count department name',
 'available department',
 'Vision of Mission college',
 'vision and mission of college ',
 'established year of college',
 'when college is started',
 'principal name of college',
 'about principal',
 'library information',
 'information about library',
 'information of library',
 'placement information 2021 and 2022',
 'about placement',
 'name of vice principal',
 'about vice principal',
 'how many labs in computer department',
 'Number of labs in computer department',
 'count of computer depatment lab']

In [18]:
def words_to_list(s):
    a=[]
    ns=""
    s=s+" "
    for i in range(len(s)):
        if s[i]==" ":
            a.append(ns)
            ns=""
        else:
            ns=ns+s[i]
    a=list(set(a))
    return a

In [19]:
def json_to_dictionary(data):
    dict=[]
    fil_dic=[]
    voc=[]
    for i in data["intents"]:
        for pattern in i["patterns"]:
            voc.append(pattern.lower())
    for i in voc:
        dict.append(words_to_list(i))
    for i in range(len(dict)):
        for word in dict[i]:
            fil_dic.append(word)
    return list(set(fil_dic))

In [20]:
chatbot_vocabulary = json_to_dictionary(data)
import difflib
def word_checker(s):
    correct_string = ""
    for word in s.casefold().split():
        if word not in chatbot_vocabulary:
            suggestion = difflib.get_close_matches(word,chartbot_vocabulary)
            for x in suggestion:
                pass
            if len(suggestion)==0:
                pass
            else:
                correct_string = correct_string+" "+str(suggestion[0])
        else:
            correct_string = correct_string+" "+str(word)
    return correct_string

In [21]:
r = sr.Recognizer()
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('rate', 170)
engine.setProperty('volume', 0.5)
engine.setProperty('voice', 'english+f5')


def bot_speaking(message):
    engine.say(message)
    engine.runAndWait()
    if engine._inLoop:
        engine.endLoop()

def get_input():
    with sr.Microphone() as source:
        bot_speaking("Hey mates Say Somthing")
        audio = r.listen(source,timeout=0)
        bot_speaking("Perfect, Thanks!!")
    try:
        msg = r.recognize_google(audio)
        print("text: "+msg);
        bot_speaking("you said "+ msg)
        return  msg
    except:
        bot_speaking("Sorry mate!! its not working")

In [22]:
def chat():
    print("Snjebia I am your college assistant and I am here to answer the query on snjb")
    while True:
        inp = input("You: ")
        if inp.lower()=="quit" or inp == None:
            break
        inp_x = word_checker(inp)

        results = model.predict([bag_of_words(inp_x,words)])[0]

        results_index = numpy.argmax(results)

        tag = label[results_index]

        if results[results_index]>=0.9:
            for tg in data["intents"]:
                if tg['tag']==tag:
                    responses = tg['response']
                    ms = random.choice(responses)
                    print("Snjbia: "+ms)
                    bot_speaking(ms)

        else:
            print("Snjbia: Sorry")
            bot_speaking("Sorry")

chat()

Snjebia I am your college assistant and I am here to answer the query on snjb
You: hi


NameError: name 'model' is not defined